### GX Core Workflow - Using Sqlite Datasource

- Import Required Libraries
- Create GX Context (Ephemeral)
- Create DataSource (Sqlite DB)
- Create Data Asset (Sqlite Table)
- Create Batch Definition 
- Create Expectations
- Create Expectations Suite and Add expectations
- Create Validation Definition (Expectation Suite, Batch Definition)
- Create Checkpoint (Validation Definition)
- Define Runtime Expectation parameters
- Run Checkpoint(Expectations parameters)
- Print Result

### Import Libraries

In [1]:
import great_expectations as gx

### Creating GX Context

In [2]:
context = gx.get_context(mode="ephemeral")


### Creating Datasource 

In [3]:
datasource_name = "sqlite_datasource"
data_source = context.data_sources.add_sqlite(
    name=datasource_name,
    connection_string="sqlite:///temperature.db"
)

### Creating Data Asset

In [4]:
data_asset_name = 'temperature_reading'
data_asset = data_source.add_table_asset(name = data_asset_name, table_name ='temperature_reading')

### Creating Batch Definition

In [5]:
batch_definition_name ='full_batch'
batch_definition = data_asset.add_batch_definition_whole_table(batch_definition_name)

### Creating Expectations

In [6]:
expectation_temp = gx.expectations.ExpectColumnValuesToBeBetween(column="temperature", 
                                                            max_value={"$PARAMETER": "temp_max_value"},
                                                            min_value={"$PARAMETER": "temp_min_value"}, catch_exceptions=True)

In [7]:
expectation_city = gx.expectations.ExpectColumnDistinctValuesToBeInSet(column="city", 
                                                            value_set={"$PARAMETER": "city_list"}, catch_exceptions=True)


### Creating Expectation Suite and Adding expectations

In [8]:
expectation_suite_name = "temperature_data_suite"
expectation_suite_ref = gx.ExpectationSuite(name=expectation_suite_name)
expectation_suite = context.suites.add(expectation_suite_ref)


In [9]:
#Adding Expectation to ExpectationSuite    
expectation_suite.add_expectation(expectation_temp)
expectation_suite.add_expectation(expectation_city)


ExpectColumnDistinctValuesToBeInSet(id='f5db1c4a-00ca-457d-8858-de3212fb5472', meta=None, notes=None, result_format=<ResultFormat.BASIC: 'BASIC'>, description=None, catch_exceptions=True, rendered_content=None, windows=None, batch_id=None, column='city', row_condition=None, condition_parser=None, value_set={'$PARAMETER': 'city_list'})

### Creating Validation Defintions

In [10]:
validation_def_name = "temperature_data_validation"
validation_definition_ref = gx.ValidationDefinition( data=batch_definition,
                                             suite=expectation_suite,
                                             name=validation_def_name)


In [11]:
validation_definition = context.validation_definitions.add(validation_definition_ref)

### Creating Checkpoint 

In [12]:
checkpoint_name = "temperature_run_checkpoint"
checkpoint_to_add = gx.Checkpoint(name=checkpoint_name,                                 
                                  validation_definitions=[validation_definition],
                                  actions=[],
                                  result_format="BASIC"
)


In [13]:
checkpoint = context.checkpoints.add(checkpoint_to_add)


### Creating Runtime Parameter Object

In [14]:
runtime_parameters = {
    "temp_min_value": 5,
    "temp_max_value":50,
    "city_list": ["Delhi", "Mumbai"],
}


### Running Checkpoint 

In [15]:
validation_results = checkpoint.run(expectation_parameters =runtime_parameters)


Calculating Metrics:   0%|          | 0/12 [00:00<?, ?it/s]

### Printing Result

In [16]:
print(validation_results)

run_id={"run_name": null, "run_time": "2025-06-23T16:27:00.439557+06:00"} run_results={ValidationResultIdentifier::temperature_data_suite/__none__/20250623T102700.439557Z/sqlite_datasource-temperature_reading: {
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_between",
        "kwargs": {
          "batch_id": "sqlite_datasource-temperature_reading",
          "column": "temperature",
          "min_value": 5.0,
          "max_value": 50.0
        },
        "meta": {},
        "id": "2cc0a62f-c4fe-4b59-a3f9-6f2b2f478828"
      },
      "result": {
        "element_count": 62,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_percent_total": 0.0,
        "unexpected_percent_nonmissing": 0.0
      },
      "meta": {},
      "exception_info": {
        "raised_excepti